# Проект Автодополнение текстов

Вы работаете в соцсетевом приложении, где пользователи постят короткие тексты. В продукте стоит задача — добавить возможность автодополнения текстов. Разработчики просят вас создать модель, которую можно запускать на мобильных устройствах. Для смартфонов есть значительные требования по оперативной памяти и скорости работы, так что важна легковесность модели.

## Постановка задачи
**Создать нейросеть, которая на основе начала фразы предсказывает её продолжение**.

Поэтапное описание задачи:
1. Взять датасет от разработчиков, очистить его, подготовить для обучения модели.
1. Реализовать и обучить модель на основе рекуррентных нейронных сетей.
1. Замерить качество разработанной и обученной модели.
1. Взять более «тяжёлую» предобученную модель из Transformers и замерить её качество.
1. Проанализировать результаты и **дать рекомендации** разработчикам: стоит ли **использовать лёгкую модель или** лучше постараться поработать с ограничениями по памяти и **использовать большую предобученную**.

## Критерии успеха
- Используемые метрики качества: ROUGE-1 и ROUGE-2
- Минимальных порогов качества модели в задаче не задано


## Описание данных

датасет с короткими постами sentiment140

https://code.s3.yandex.net/deep-learning/tweets.txt

# Этап 0. Подготовка окружения

Актуальный код функций доступен тут:

https://github.com/evgeny-ydsc/DLE-NLP

In [2]:
!pip install evaluate -q
!pip install rouge_score -q

In [3]:
IS_GOOGLE_COLAB=True

In [4]:
import sys
import evaluate

if IS_GOOGLE_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    sys.path.append('/content/drive/MyDrive/project1')
else:
    sys.path.append('./')

from src.next_token_dataset import prepare_loaders
from src.lstm_model import LSTMGenerator
from src.lstm_train_eval import ModelTrainer
from src.data_utils import download_dataset
from src.eval_transformer_pipeline import Distilgpt2Generator

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Этап 1. Сбор и подготовка данных

In [5]:
if IS_GOOGLE_COLAB:
    MAX_TEXTS_COUNT = 100_000
else: # просто для черновой отладки локально на CPU
    MAX_TEXTS_COUNT = 100


In [7]:
if IS_GOOGLE_COLAB:
    raw_data_path = "drive/MyDrive/project1/data/tweets.txt"
else: # просто для черновой отладки локально на CPU
    raw_data_path = "data/tweets.txt"

download_dataset("https://code.s3.yandex.net/deep-learning/tweets.txt",
                                 raw_data_path)
train_loader, val_loader, test_loader, tokenizer, val_texts, test_texts = \
    prepare_loaders (raw_data_path, batch_size=256, max_texts_count=MAX_TEXTS_COUNT)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Этап 2. Реализация рекуррентной сети


In [8]:
model = LSTMGenerator(tokenizer.vocab_size)

# Этап 3. Тренировка модели

In [9]:
trainer = ModelTrainer (model, tokenizer)
trainer.train(train_loader, val_loader, val_texts, n_epochs=5)

Evaluating ROUGE: 100%|██████████| 10000/10000 [01:41<00:00, 98.95it/s] 


Epoch 1 | Train Loss: 6.028 | Val Loss: 5.644 | Val Accuracy: 15.77%| rouge1: 0.039 | rouge2: 0.004


Evaluating ROUGE: 100%|██████████| 10000/10000 [01:40<00:00, 99.12it/s]


Epoch 2 | Train Loss: 5.344 | Val Loss: 5.531 | Val Accuracy: 16.95%| rouge1: 0.035 | rouge2: 0.004


Evaluating ROUGE: 100%|██████████| 10000/10000 [01:41<00:00, 98.61it/s] 


Epoch 3 | Train Loss: 5.066 | Val Loss: 5.549 | Val Accuracy: 17.44%| rouge1: 0.044 | rouge2: 0.004


Evaluating ROUGE: 100%|██████████| 10000/10000 [01:40<00:00, 99.31it/s]


Epoch 4 | Train Loss: 4.868 | Val Loss: 5.592 | Val Accuracy: 17.73%| rouge1: 0.038 | rouge2: 0.004


Evaluating ROUGE: 100%|██████████| 10000/10000 [01:41<00:00, 98.77it/s]


Epoch 5 | Train Loss: 4.711 | Val Loss: 5.650 | Val Accuracy: 17.83%| rouge1: 0.033 | rouge2: 0.004


~~Видно, что начиная с 5й эпохи модель начала переобучатсья. Но GPU Практикума недоступна, а ресурс colab тоже уже закончился и не гарантирован (уже обрывался по полпути ранее). Поэтому не рискую его перезапускать ещё раз, оставлю модель как есть в этот раз.~~

In [10]:
rouge = evaluate.load("rouge")
for line in test_texts[:3]:
    l = len(line)
    i = int(l*0.75)
    start = line [:i]
    finish = line [i:]
    predicted_finish = model.generate_output_text(tokenizer, start, l-i)
    results = rouge.compute(predictions=[predicted_finish], references=[finish])
    print (f"----------------------------------------------")
    print (f"оригинальная строка: {line}")
    print (f"оригинальное начало: {start}")
    print (f"оригинальный конец : {finish}")
    print (f"предсказанный конец: {predicted_finish}")
    print (f"rouge1={results['rouge1']:.3f}, rouge2={results['rouge2']:.3f}\n")


----------------------------------------------
оригинальная строка: addicted to dramasi have to stop watching them continuallymake self tired and not doing things have to do
оригинальное начало: addicted to dramasi have to stop watching them continuallymake self tired and 
оригинальный конец : not doing things have to do
предсказанный конец: i have to go to work tomorrow morning and i have to go to work tomorrow morning and i have to go to work tomorrow morning and
rouge1=0.121, rouge2=0.065

----------------------------------------------
оригинальная строка: mesirii no adhoc meeting spontaneously turned into code camp
оригинальное начало: mesirii no adhoc meeting spontaneously turned
оригинальный конец :  into code camp
предсказанный конец: out to the beach and i have to go to work tomorrow morning and i
rouge1=0.000, rouge2=0.000

----------------------------------------------
оригинальная строка: i wish my computer wasnt broken so i could get a effin pic upwhy cant i do it on my bla

# Этап 4. Использование предобученного трансформера

In [11]:
bert_model = Distilgpt2Generator()

Device set to use cuda:0


div class="alert alert-info" style="background-color:#FFF8DC;color:black;">
<b>🎓 Комментарии студента v1:</b>

Нашёл ошибку в коде с трансформером - строка обрезалась дважды - и в функции generate_output_text, и в сниппете ниже [l:] - поправил.

Поэтому и выдача трансформера была такая странная.

In [12]:
predictions=[]
references=[]

for line in test_texts[:3]:
    l = len(line)
    i = int(l*0.75)
    start = line [:i]
    finish = line [i:]
    predicted_finish = bert_model.generate_output_text(start, l-i)#v1[l:]
    results = rouge.compute(predictions=[predicted_finish], references=[finish])
    print (f"----------------------------------------------")
    print (f"оригинальная строка: {line}")
    print (f"оригинальное начало: {start}")
    print (f"оригинальный конец : {finish}")
    print (f"предсказанный конец: {predicted_finish}")
    print (f"rouge1={results['rouge1']:.3f}, rouge2={results['rouge2']:.3f}")

    predictions.append(predicted_finish)
    references.append(finish)

results = rouge.compute(predictions=predictions, references=references)
print (f"rouge1={results['rouge1']:.3f}, rouge2={results['rouge2']:.3f}\n")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=27) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=15) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/te

----------------------------------------------
оригинальная строка: addicted to dramasi have to stop watching them continuallymake self tired and not doing things have to do
оригинальное начало: addicted to dramasi have to stop watching them continuallymake self tired and 
оригинальный конец : not doing things have to do
предсказанный конец: icky.
rouge1=0.000, rouge2=0.000


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=24) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


----------------------------------------------
оригинальная строка: mesirii no adhoc meeting spontaneously turned into code camp
оригинальное начало: mesirii no adhoc meeting spontaneously turned
оригинальный конец :  into code camp
предсказанный конец:  into a violent clash.



























































































































































































































































rouge1=0.286, rouge2=0.000
----------------------------------------------
оригинальная строка: i wish my computer wasnt broken so i could get a effin pic upwhy cant i do it on my blackberrry
оригинальное начало: i wish my computer wasnt broken so i could get a effin pic upwhy cant i
оригинальный конец :  do it on my blackberrry
предсказанный конец:  make my computer. I have been able to do it for over a year now and dont have any problems with it. I think i will alway

# Этап 5. Формулирование выводов

Была обучена модель на основе LSTM. Показатели на лучшем тексте из трёх: rouge1=0.121, rouge2=0.065

Также был использован предобученный трансформер, усреднённые показатели: rouge1=0.149, rouge2=0.019

Из этого следует, что использование предобученного трансформера предпочтительнее полученной LSTM-модели.